In [1]:
%cd ..

/home/soda/rcappuzz/work/benchmark-join-suggestions


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pickle
import polars as pl

In [4]:
import src.table_integration.join_profiling as jp

In [17]:
with open("generated_candidates.pickle", "rb") as fp:
    generated_candidates = pickle.load(fp)

In [6]:
one_candidate = generated_candidates["minhash"].popitem()
hash_, mdata = one_candidate
source_metadata = mdata.source_metadata
candidate_metadata = mdata.candidate_metadata
source_table = pl.read_parquet(source_metadata["full_path"])
candidate_table = pl.read_parquet(candidate_metadata["full_path"])
left_on = mdata.left_on
right_on = mdata.right_on


In [7]:
unique_source = jp.find_unique_keys(source_table, left_on)
unique_cand = jp.find_unique_keys(candidate_table, right_on)

In [8]:
jp.measure_cardinality_proportion(source_table, candidate_table)

1.9781387834649709

The cardinality proportion is larger than 1!!! This is not something they are accounting for in the heuristic. 

How are we handling these situations? Should we go for `min([card_source/card_cand, card_cand/card_source])` instead? Is this even meaningful? 

I am having issues here to find the containment 

In [9]:
unique_source = source_table[left_on].unique()
unique_cand = candidate_table[right_on].unique()
intersection=source_table.join(unique_cand, left_on=left_on, right_on=right_on, how="inner")
containment = len(intersection)/len(source_table)
print(containment)

0.510192573195315


In [10]:
jp.measure_join_quality(source_table, candidate_table, left_on, right_on)

4

In [13]:
%%timeit
jp.measure_containment(unique_source, unique_cand, left_on=left_on, right_on=right_on)

167 µs ± 193 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [12]:
%%timeit
s1 = set(unique_source.to_series())
s2 = set(unique_cand.to_series())
len(s1.intersection(s2))/len(unique_source)

507 µs ± 41.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [76]:
on = ["subject", "isLocatedIn"]
left_on = right_on = on
source_table = source_table[left_on].unique()
unique_cand = candidate_table[right_on].unique()
# jp.measure_containment(unique_source, unique_cand, on, on)

In [109]:
l_pd = source_table[left_on].drop_nulls().to_pandas()
r_pd = candidate_table[right_on].drop_nulls().to_pandas()

In [111]:
len(l_pd) * len(r_pd)

291489051

In [15]:
jp.estimate_join_size(source_table, candidate_table, left_on, right_on)

[30773728.30690537, 31253318.877922077]

In [39]:
for hash_, mdata in generated_candidates["minhash"].items():
    source_metadata = mdata.source_metadata
    candidate_metadata = mdata.candidate_metadata
    source_table = pl.read_parquet(source_metadata["full_path"])
    candidate_table = pl.read_parquet(candidate_metadata["full_path"])
    left_on = mdata.left_on
    right_on = mdata.right_on
    print("Estimated join size")
    estimates = jp.estimate_join_size(source_table, candidate_table, left_on, right_on)
    print(estimates)
    print("Join quality")
    print(jp.measure_join_quality(source_table, candidate_table, left_on, right_on))
    print("Containment")
    print(jp.measure_containment(source_table, candidate_table, left_on, right_on))
    print("Cardinality proportion")
    print(jp.measure_cardinality_proportion(source_table, candidate_table))
    if min(estimates) < 1e6:
        joined =  jp.merge_table(source_table, candidate_table, left_on=left_on, right_on=right_on, how="inner")
        print("True join size: ", len(joined))
    break

Estimated join size
[417855.3989769821, 2513560.9384615384]
Join quality
1
Containment
0.16624040920716113
Cardinality proportion
145.68366383380547
True join size:  19721973


In [36]:
unique_source = jp.find_unique_keys(source_table, left_on)
unique_cand = jp.find_unique_keys(candidate_table, right_on)


In [38]:
len(source_table[left_on].drop_nulls()) * len(candidate_table[right_on].drop_nulls()) / len(unique_source)

213200.26342710998

In [28]:
19721973-163381461

-143659488

In [29]:
len(source_table)

154279

In [31]:
candidate_table

type,subject,isLocatedIn,happenedOnDate
str,str,str,str
"""<wordnet_movie…","""<Justice_(1914…","""<United_Kingdo…","""2016-05-##^^xs…"
"""<wordnet_movie…","""<Timecode_(200…","""<United_States…","""2014-##-##^^xs…"
"""<wordnet_movie…","""<Khwab_Ki_Duni…","""<India>""","""2015-10-##^^xs…"
"""<wordnet_movie…","""<Nursie!_Nursi…","""<United_Kingdo…","""2016-06-##^^xs…"
"""<wordnet_movie…","""<Dr._Wake's_Pa…","""<United_Kingdo…","""2016-06-##^^xs…"
"""<wordnet_movie…","""<...La_bocca_m…","""<Italy>""","""2019-05-##^^xs…"
"""<wordnet_movie…","""<Old_San_Franc…","""<United_States…","""1906-##-##^^xs…"
"""<wordnet_movie…","""<Hum_Bhi_Insaa…","""<India>""","""2015-10-##^^xs…"
"""<wordnet_movie…","""<Your_Place_or…","""<United_States…","""2021-08-##^^xs…"
